In [5]:
#Utils
from tqdm.notebook import tqdm
from ml_utils import *
from machine_learning_models import *
from fingerprints import *
import deepchem as dc
from deepchem.models import GraphConvModel

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [6]:
# Load CCR results path
ccr_path = "./ccr_results/"
# Load actives dB
db_path = './dataset/'
#Load active db
regression_db = pd.read_csv(os.path.join(db_path, f'chembl_30_IC50_10_tids_1000_CPDs.csv'))
# Target Classes
regression_tids = regression_db.chembl_tid.unique()[:]

# Models Parameters
### Select the desired parameters to be used by regression models
<p>
<li> <b>model_list</b>: ML/DL models for regression (kNN: k-neirest neighbor, SVR: Support Vector Regression, RFR: Random Forest Regression, DNN: Deep Neural Network, MR: Median regression)</li>
</p>
<p>
<li> <b>compound_sets</b>: Compound sets to be generated ('Cluster set': Largest Analogue series, ' Potent set': Most potent compounds) </li>
</p>
<p>
<li> <b>potent_size</b>: Potent sets size to be generated (0.1 = 10% original set) </li>
</p>

<p>
<li> <b>params_dict</b>: GCN hyperparameter grid (nb_epoch: number of epochs, learning_rate, graph_conv_layer, dense_layer_size, dropout, number of atom features) </li>
</p>


In [9]:
model_list = ['GCN']
compound_sets = ['Potent set', 'Cluster set']
potent_size = 0.1
params_dict = {
    "nb_epoch":[100, 200],
    "learning_rate":[0.01, 0.001],
    "n_tasks":[1],
    "graph_conv_layers":[[64, 64], [256, 256], [512, 512], [1024, 1024]],
    "dense_layer_size":[64, 256, 512, 1024],
    "dropout":[0.0],
    "mode":["regression"],
    "number_atom_features":[75],}

In [ ]:
performance_train_df = pd.DataFrame()
performance_test_df = pd.DataFrame()
predictions_test_df = pd.DataFrame()
predictions_train_df = pd.DataFrame()
parameter_resume = []

for target in tqdm(regression_tids):
    for approach in compound_sets:
            print(f'Training on {target} {approach}')

            # Select Target Database
            regression_db_tid = regression_db.loc[regression_db.chembl_tid == target]

            if approach == 'Cluster set':

                #Load CCR database
                ccr_df = pd.read_csv(os.path.join(ccr_path, f'CCR_C30_IC50_HT_single_5_0.666_13_{target}.csv'))

                #Load largest analogue series
                ccr_df_AS = ccr_df.loc[ccr_df['Core'] == ccr_df['Core'].value_counts().index[0]].chembl_id.values

                # Select Training and Test datasets
                df_TR = regression_db_tid.loc[~regression_db_tid['chembl_cid'].isin(ccr_df_AS)]
                df_TE = regression_db_tid.loc[regression_db_tid['chembl_cid'].isin(ccr_df_AS)]

            elif approach == 'Potent set':

                # Select Training and Test datasets
                df_TE = regression_db_tid.nlargest(int(round(len(regression_db_tid.index)*potent_size, 0)), 'pPot')
                df_TR = regression_db_tid.loc[~regression_db_tid['chembl_cid'].isin(df_TE['chembl_cid'])]

            # Generate Mol object from SMILES
            mols_tr = [Chem.MolFromSmiles(smi) for smi in df_TR.nonstereo_aromatic_smiles.tolist()]
            mols_te = [Chem.MolFromSmiles(smi) for smi in df_TE.nonstereo_aromatic_smiles.tolist()]

            # Constructing mol featurizer
            featurizer = dc.feat.ConvMolFeaturizer()
            mol_graphs_tr = featurizer.featurize(mols_tr)
            mol_graphs_te = featurizer.featurize(mols_te)
            mol_graphs = np.concatenate((mol_graphs_tr, mol_graphs_te))

            # Potency values
            potency = np.concatenate((np.array(df_TE.pPot.values), np.array(df_TR.pPot.values)))
            # Compound Ids
            ids = np.concatenate((np.array(df_TR.chembl_tid.values), np.array(df_TE.chembl_tid.values)))

            # Constructing Deepchem Datasets
            dataset = dc.data.NumpyDataset(X=mol_graphs, y=potency, ids=ids)
            training_set_u = dc.data.NumpyDataset(X=mol_graphs_tr, y=np.array(df_TR.pPot.values), ids=np.array(df_TR.chembl_tid.values))
            test_set_u = dc.data.NumpyDataset(X=mol_graphs_te, y=np.array(df_TE.pPot.values), ids=np.array(df_TE.chembl_tid.values))

            # Initialize transformers
            transformers = [dc.trans.NormalizationTransformer(transform_y=True, dataset=dataset, move_mean=False)]

            #Transform data
            for transformer in transformers:
                training_set = transformer.transform(training_set_u)
            for transformer in transformers:
                test_set = transformer.transform(test_set_u)

            for trial in range(1):

                # Split dataset into TR and internal Validation
                splitter = dc.splits.RandomSplitter()
                train_set, valid_set = splitter.train_test_split(training_set, seed=trial)

                print(f'Starting Trial {trial}')
                for model in model_list:

                    #Define random seed
                    set_seeds(trial)

                    #Initialize GridSearch optimizer
                    optimizer = dc.hyper.GridHyperparamOpt(dc.models.GraphConvModel)

                    # Select optimization metric (MAE)
                    metric = dc.metrics.Metric(dc.metrics.mae_score)

                    # Best GCN model, parameters and final results
                    best_model, best_params, all_results = optimizer.hyperparam_search(params_dict=params_dict,
                                                                                       train_dataset=train_set,
                                                                                       valid_dataset=valid_set,
                                                                                       metric=metric,
                                                                                       use_max=False,
                                                                                       output_transformers=transformers,
                                                                                       #logdir=r'C:\\GCN\\'
                                                                                       )
                    # Define final GCN model
                    def final_gcn(data, best_params):

                        gcn = GraphConvModel(n_tasks=best_params["n_tasks"],
                                       graph_conv_layers=best_params["graph_conv_layers"],
                                       dropout=best_params["dropout"],
                                        mode=best_params["mode"],
                                       predictor_hidden_feats=best_params["dense_layer_size"],
                                       learning_rate=best_params["learning_rate"],
                                       )

                        gcn.fit(data, nb_epoch=best_params["nb_epoch"])

                        return gcn

                    #Best model parameters
                    opt_parameters_dict = {'model': model,
                                           'trial': trial,
                                           'Target ID': target,
                                           'Approach':approach}

                    if isinstance(best_params, tuple):
                        best_params = {
                            "nb_epoch":best_params[0],
                            "learning_rate":best_params[1],
                            "n_tasks":best_params[2],
                            "graph_conv_layers":best_params[3],
                            "dense_layer_size":best_params[4],
                            "dropout":best_params[5],
                            "mode":best_params[6],
                            "number_atom_features":best_params[7]}

                    for param, value in best_params.items():
                        opt_parameters_dict[param] = value
                    parameter_resume.append(opt_parameters_dict)

                    # Generate final Model
                    ml_model = final_gcn(training_set, best_params)

                    # evaluate the model
                    train_score = ml_model.evaluate(training_set, [metric], transformers)
                    test_score = ml_model.evaluate(test_set, [metric], transformers)

                    #TRAIN
                    #Model Evaluation
                    model_eval_train = Model_Evaluation(ml_model, training_set, training_set_u.y, transformers[0], model_id=model)

                    #Performance df
                    performance_train = model_eval_train.pred_performance
                    performance_train["trial"] = trial
                    performance_train["Approach"] = approach
                    performance_train_df = pd.concat([performance_train_df, performance_train])

                    # Prediction df
                    predictions_train = model_eval_train.predictions
                    predictions_train["trial"] = trial
                    predictions_train["Approach"] = approach
                    predictions_train_df = pd.concat([predictions_train_df, predictions_train])

                    #Model Evaluation
                    model_eval_test = Model_Evaluation(ml_model, test_set, test_set_u.y, transformers[0], model_id=model)

                    #Performance df
                    performance_test = model_eval_test.pred_performance
                    performance_test["trial"] = trial
                    performance_test["Approach"] = approach
                    performance_test_df = pd.concat([performance_test_df, performance_test])

                    # Prediction df
                    predictions_test = model_eval_test.predictions
                    predictions_test["trial"] = trial
                    predictions_test["Approach"] = approach
                    predictions_test_df = pd.concat([predictions_test_df, predictions_test])

                    del best_model, best_params, all_results, ml_model

parameter_df = pd.DataFrame(parameter_resume)

# Save Final Dataframes
result_path = create_directory('./regression_results/cluster_potent/')
performance_train_df.to_csv(os.path.join(result_path, f'performance_train_gcn_cluster_potent.csv'))
performance_test_df.to_csv(os.path.join(result_path, f'performance_test_gcn_cluster_potent.csv'))
parameter_df.to_csv(os.path.join(result_path, f'model_best_parameters_gcn_cluster_potent.csv'))
predictions_test_df.to_csv(os.path.join(result_path, f'predictions_test_gcn_cluster_potent.csv'))
predictions_train_df.to_csv(os.path.join(result_path, f'predictions_train_gcn_cluster_potent.csv'))